# Packing circuits

This notebook looks at some examples of packing circuits to reduce their ancilla costs. In particular in these examples this problem is solved via the calculation of minimum vertex covers, which is done by not allowing 'evicted gates'.

First, import the modules we need.

In [1]:
from pytket import Circuit
from pytket.circuit.display import render_circuit_jupyter
from pytket_dqc.networks import NISQNetwork
from pytket_dqc.distributors import GraphPartitioning, Annealing
from pytket_dqc.circuits import DistributedCircuit, BipartiteCircuit
from pytket.transform import Transform
from pytket_dqc.utils.gateset import dqc_rebase

## Trivial examples

### Two CZs

In [2]:
circuit0 = Circuit(2)
circuit0.CZ(0,1)
circuit0.CZ(0,1)
dqc_rebase.apply(circuit0)
network0 = NISQNetwork(
    [[0,1]],
    {0: [0], 1: [1]}
)
render_circuit_jupyter(circuit0)

dist_circ0 = DistributedCircuit(circuit0)
distributor = GraphPartitioning()
placement0 = distributor.distribute(dist_circ0, network0)

In [3]:
bp_circuit0 = BipartiteCircuit(circuit0, placement0)
render_circuit_jupyter(bp_circuit0.packed_circuit)

AssertionError: 
                    This vertex has no link_qubit associated with it.
                    Vertex register:<pytket_dqc.packing.packing.ExtendedRegister object at 0x17a9f7700>
                    Qubit: Server 1[0]
                    Linked Register: <pytket_dqc.packing.packing.ExtendedRegister object at 0x17a9f7730>
                    N commands: 11
                    N added commands: 1
                    

### Two CZs (opposite controls)

In [ ]:
circuit1 = Circuit(2)
circuit1.CZ(0,1)
circuit1.CZ(1,0)
network1 = NISQNetwork(
    [[0,1]],
    {0: [0], 1: [1]}
)
render_circuit_jupyter(circuit1)

dist_circ1 = DistributedCircuit(circuit1)
distributor = GraphPartitioning()
placement1 = distributor.distribute(dist_circ1, network1)

In [ ]:
bp_circuit1 = BipartiteCircuit(circuit1, placement1)
render_circuit_jupyter(bp_circuit1.packed_circuit)

### Two CZs with X in between

In [ ]:
circuit2 = Circuit(2)
circuit2.CZ(0,1)
circuit2.Rx(1, 0)
circuit2.Rx(1, 1)
circuit2.CZ(1,0)
network2 = NISQNetwork(
    [[0,1]],
    {0: [0], 1: [1]}
)
render_circuit_jupyter(circuit2)

dist_circ2 = DistributedCircuit(circuit2)
distributor2 = GraphPartitioning()
placement2 = distributor.distribute(dist_circ2, network2)

In [ ]:
bp_circuit2 = BipartiteCircuit(circuit2, placement2)
render_circuit_jupyter(bp_circuit2.packed_circuit)

### Two CZs with non-packable in between

In [ ]:
circuit3 = Circuit(2)
circuit3.CZ(0,1)
circuit3.Rx(0.5, 0)
circuit3.Rx(0.5, 1)
circuit3.CZ(1,0)
network3 = NISQNetwork(
    [[0,1]],
    {0: [0], 1: [1]}
)
render_circuit_jupyter(circuit3)

dist_circ3 = DistributedCircuit(circuit3)
distributor = GraphPartitioning()
placement3 = distributor.distribute(dist_circ3, network3)

In [ ]:
bp_circuit3 = BipartiteCircuit(circuit3, placement3)
render_circuit_jupyter(bp_circuit3.packed_circuit)

### Simple CZ cycle

In [ ]:
circuit4 = Circuit(3)
circuit4.CZ(0, 1)
circuit4.CZ(1, 2)
circuit4.CZ(2, 0)
network4 = NISQNetwork(
    [[0,1], [1, 2]],
    {0: [0], 1: [1], 2:[2]}
)
render_circuit_jupyter(circuit4)

dist_circ4 = DistributedCircuit(circuit4)
distributor = GraphPartitioning()
placement4 = distributor.distribute(dist_circ4, network4)

In [ ]:
bp_circuit4 = BipartiteCircuit(circuit4, placement4)
render_circuit_jupyter(bp_circuit4.packed_circuit)

### Complicated circuit

In [ ]:
circuit5 = Circuit(6)
circuit5.CZ(0, 3).CZ(2, 3).CZ(2, 4).H(2).CZ(2, 5).CZ(2, 4).CZ(0, 3).CZ(0, 4).H(3).CZ(0, 5).X(0).CZ(0, 3)
Transform.RebaseToQuil().apply(circuit5)
network5 = NISQNetwork([[0,1]], {0:[0,1,2], 1:[3,4,5]})
render_circuit_jupyter(circuit5)

dist_circ5 = DistributedCircuit(circuit5)
distributor = GraphPartitioning()
placement5 = distributor.distribute(dist_circ5, network5)

In [ ]:
bp_circuit5 = BipartiteCircuit(circuit5, placement5)
render_circuit_jupyter(bp_circuit5.packed_circuit)